In [1]:
using Parameters
using Lattices
using LinearAlgebra 
using Statistics
using Lattices
using Random
using TAU
using LatticeModels
using Arpack
using LogExpFunctions
using Plots
plotly()

┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots /Users/kimyeongjun/.julia/packages/Plots/3E7jF/src/backends.jl:318


Plots.PlotlyBackend()

In [2]:
dropmean(A; dims=:) = dropdims(mean(A; dims=dims); dims=dims)

dropmean (generic function with 1 method)

In [3]:
function box_indices(ltc::Lattice2D, b::Integer)
    @assert ltc.N%b == 0
    L = ltc.N
    U = ltc.U
    box_pts = U*b^2
    box_num = L^2÷b^2
    inds = Array{Int64}(undef, box_pts, box_num)
    box_count=1
    for x in 1:b:L-b+1, y in 1:b:L-b+1
        idx_count = 1
        for m in 0:b-1, n in 0:b-1, u in 1:U
            inds[idx_count, box_count] = index(ltc, (x+m,y+n, u))
            idx_count += 1
        end
        box_count += 1
    end
    return inds
end

"""
Return box indices with box size b. First dimension is the box indices, second dimension is different boxes
"""
function box_indices(ltc::Lattice3D, b::Integer)
    @assert ltc.N%b == 0 && ltc.L == ltc.M == ltc.N
    L = ltc.N
    U = ltc.U
    box_pts = U*b^3
    box_num = L^3÷b^3
    inds = Array{Int64}(undef, box_pts, box_num)
    box_count=1
    for x in 1:b:L-b+1, y in 1:b:L-b+1, z in 1:b:L-b+1
        idx_count = 1
        for l in 0:b-1, m in 0:b-1, n in 0:b-1, u in 1:U
            inds[idx_count, box_count] = index(ltc, (x+l,y+m,z+n, u))
            idx_count += 1
        end
        box_count += 1
    end
    return inds
end

function box_coarse(p, box_inds)
    @assert size(p, 1) == length(box_inds)
    p_coarse  = Array{eltype(p)}(undef, size(box_inds, 2), size(p, 2))
    for j in 1:size(p, 2), i in 1:size(box_inds, 2)
        p_coarse[i, j] = sum(p[box_inds[:, i], j])
    end

    if typeof(p) <: Vector
        return vec(p_coarse)
    else
        return p_coarse
    end
end

box_coarse (generic function with 1 method)

In [4]:
struct MFAParameters{Lat <: Lattice}
    ltc::Lat
    l::Vector{Int64}
    q::Vector{Float64}
    box_indices::Vector{Array{Int64, 2}}
    function MFAParameters(ltc::Lat, l, q) where {Lat <: Lattice} 
        new{Lat}(ltc, l, q, Vector{Array{Int64, 2}}[]) 
    end
end

function prepare_MFA!(params::MFAParameters)
    for li in params.l
       idx_li = box_indices(params.ltc, li)
       push!(params.box_indices, idx_li)
    end
end

prepare_MFA! (generic function with 1 method)

In [140]:
function compute_fα(p, L, l, q)
    μ = p.^q
    Z_q = sum(μ)
    for i in 1:size(p, 2)
        μ[:, i] ./= Z_q[i]
    end
    return (1/log(l/L))*sum(xlogx.(μ))
end

function compute_α(p, L, l, q)
    μ = p.^q
    Z_q = sum(μ)
    for i in 1:size(p, 2)
        μ[:, i] ./= Z_q[i]
    end
    return (1/log(l/L))*sum(xlogy.(μ, p))
end

function compute_fα(p, params::MFAParameters)
    p_coarse = [box_coarse(p, params.box_indices[i]) for i in 1:length(params.l)]
    fα = [compute_fα(p_coarse[i], L, params.l[i], params.q[j]) for i in 1:length(params.l), j in 1:length(params.q)]
    return fα
end

function compute_α(p, params::MFAParameters; q = 2.)
    p_coarse = [box_coarse(p, params.box_indices[i]) for i in 1:length(params.l)]
    α = [compute_α(p_coarse[i], L, params.l[i], params.q[j]) for i in 1:length(params.l), j in 1:length(params.q)]
    return α
end

compute_α (generic function with 2 methods)

In [127]:
L = 20
l = [2]
l_str = reshape(["$(l[i])" for i in 1:length(l)], 1, length(l))
q = collect(-3:0.3:3)
ltc = Lattice3D(L, L, L, 1)
params = MFAParameters(ltc, l, q)
prepare_MFA!(params)
W = [2.0, 16.5, 32.0]

3-element Vector{Float64}:
  2.0
 16.5
 32.0

In [128]:
psi1 = Array{Float64}[]
psi2 = Array{Float64}[]
psi3 = Array{Float64}[]


for R in 1:1
    H1 = cubic(L = L, M = L, N = L) .+ W[1]*Diagonal(rand(L*L*L) .- 0.5)
    H2 = cubic(L = L, M = L, N = L) .+ W[2]*Diagonal(rand(L*L*L) .- 0.5)
    H3 = cubic(L = L, M = L, N = L) .+ W[3]*Diagonal(rand(L*L*L) .- 0.5)

    E, psi1 = eigs(H1, sigma = 0., nev = 2)
    E, psi2 = eigs(H2, sigma = 0., nev = 2)
    E, psi3 = eigs(H3, sigma = 0., nev = 2)
    psi1 = cat(psi1, psi1, dims = 2)
    psi2 = cat(psi2, psi2, dims = 2)
    psi3 = cat(psi3, psi3, dims = 2)
end

In [139]:
α1 = compute_α(abs2.(psi1[:, 2]), params)
α2 = compute_α(abs2.(psi2[:, 2]), params)
α3 = compute_α(abs2.(psi3[:, 2]), params)

fα1 = compute_fα(abs2.(psi1[:, 2]), params)
fα2 = compute_fα(abs2.(psi2[:, 2]), params)
fα3 = compute_fα(abs2.(psi3[:, 2]), params)

1×21 Matrix{Float64}:
 0.189555  0.227992  0.278114  0.345424  …  0.0124127  0.00610296  0.003033

In [130]:
plot(q, α1)
plot!(q, α2)
plot!(q, α3)

In [131]:
plot(α1, fα1)
plot!(α2, fα2)
plot!(α3, fα3)

In [136]:
τ1 = q*α1 .- fα1
τ2 = q*α2 .- fα2
τ3 = q*α3 .- fα3

21×21 Matrix{Float64}:
 -47.2771    -47.275     -47.2659    …  -0.333098   -0.319303    -0.312979
 -42.5684    -42.5703    -42.5671       -0.301029   -0.287983    -0.281984
 -37.8596    -37.8656    -37.8683       -0.268961   -0.256663    -0.25099
 -33.1508    -33.1609    -33.1696       -0.236892   -0.225343    -0.219995
 -28.4421    -28.4562    -28.4708       -0.204824   -0.194023    -0.189
 -23.7333    -23.7515    -23.772     …  -0.172755   -0.162703    -0.158006
 -19.0246    -19.0468    -19.0732       -0.140687   -0.131383    -0.127011
 -14.3158    -14.3421    -14.3744       -0.108618   -0.100063    -0.0960167
  -9.60707    -9.63738    -9.67567      -0.0765497  -0.068743    -0.0650221
  -4.89831    -4.93269    -4.97689      -0.0444812  -0.037423    -0.0340276
  -0.189555   -0.227992   -0.278114  …  -0.0124127  -0.00610296  -0.003033
   4.5192      4.47671     4.42066       0.0196558   0.0252171    0.0279616
   9.22796     9.1814      9.11944       0.0517242   0.0565371    0.0589561
 

In [137]:
p3 = plot(q, τ1)
plot!(p3, q, τ2)
plot!(p3, q, τ3)
plot!(q, 3*(q .- 1), line = :dot, c = :red, lw = 2)

In [138]:
println(τ1[end])
println(τ2[end])
println(τ3[end])

5.649380410740853
2.7962337658710106
0.3069126540003103
